# Imports

In [253]:
from selenium import webdriver
import re as regx
import time
import datetime
from selenium.webdriver.common.keys import Keys

# Browser

In [136]:
__BROWSER__ = webdriver.Chrome(executable_path='/Users/matthewraymond/chromedriver')
__BROWSER__.implicitly_wait(10)

# Global Testing Variables

In [137]:
__TESTINGWEBSITES__ = ["https://www.sccourts.org/casesearch/", # agreement says no crawlers
                       "https://wcca.wicourts.gov/case.html",
                       "https://www.oscn.net/dockets/search.aspx",
                       "https://www.civitekflorida.com/ocrs/county/", # recapcha
                       "https://apps.supremecourt.az.gov/publicaccess/caselookup.aspx", # captcha
                       "https://www.superiorcourt.maricopa.gov/docket/CriminalCourtCases/caseSearch.asp",
                       "http://justicecourts.maricopa.gov/FindACase/casehistory.aspx",
                       "https://casesearch.epcounty.com/PublicAccess/default.aspx"]

__LEGALWEBSITES__ = __TESTINGWEBSITES__[1:3] + __TESTINGWEBSITES__[-3:]

__TESTING_FNAME__ = "John"
__TESTING_LNAME__ = "Smith"
__TESTING_MNAME__ = "Herbert"

__TESTING_DOB__ = datetime.datetime(2001, 9, 11)

In [138]:
# Removes one for testing
__LEGALWEBSITES__ = __LEGALWEBSITES__[:-1]

# Regex

In [272]:
__FIRSTNAME__ = "=\"[^\"]*[Ff][A-Za-z]*[Nn]ame[A-Za-z]*\""
__LASTNAME__ = "=\"[^\"]*[Ll][A-Za-z]*[Nn]ame[A-Za-z]*\""
__MIDDLENAME__ = "=\"[^\"]*[Mm][A-Za-z]*[Nn]ame[A-Za-z]*\""
__BIRTHDATE__ = "=\"[^\"]*((dob)|(birth))[^\"]*\""
__DATEFORMAT__ = "[MD\d]{2}[./-][MD\d]{2}[./-][Y\d]{4}"
__REACTFORMAT__ = "\"[^\"]*?(react)[^\"]\"><\/div>"
__SEARCHBUTTON__ = "=\"[^\"]*?(search)[^\"]*?\""

# Function Declaration

## Scraping Info from Webpage

In [267]:
# function that get's the html from a web url
def getSource(strURL):
    __BROWSER__.get(strURL)
    print("Scraping website source code...")
    
    html = __BROWSER__.page_source
    while regx.search(__REACTFORMAT__, html, regx.IGNORECASE) is not None:
        time.sleep(1)
        html = __BROWSER__.page_source
          
    return html

def regSearchName(ws, re):
    _result = regx.search("name" + re, ws, regx.IGNORECASE)
    if(_result is None):
        return "";
    else:
        return _result[0][6:-1]

def regSearchID(ws, re):
    _result = regx.search("id" + re, ws, regx.IGNORECASE)
    if(_result is None):
        return "";
    else:
        return _result[0][4:-1]

def getIDandName(ws, re):
    return (regSearchID(ws, re), regSearchName(ws, re))

## Input Filling

In [291]:
def fillOutName(html, fn, ln, mn):
    fnID, fnName = getIDandName(html, __FIRSTNAME__)
    mnID, mnName = getIDandName(html, __MIDDLENAME__)
    lnID, lnName = getIDandName(html, __LASTNAME__)

    sendInput(fnName, fn)
    sendInput(mnName, mn)
    sendInput(lnName, ln)

    
def sendInput(e, s):
    item = __BROWSER__.find_element_by_name(e)
    item.click()
    item.clear()
    item.send_keys(s)
    
def fillOutDOB(html, dob):
    if dob is not None:
        dID, dName = getIDandName(html, __BIRTHDATE__)
        if dName is not "":
            item = __BROWSER__.find_element_by_name(dName)
        else:
            item = __BROWSER__.find_element_by_id(dID)
        item.click()
        item.clear()
        item.send_keys("{1}{0}{2}{0}{3}".format(findDateFormat(html), dob.month, dob.day, dob.year))
        
    
def findDateFormat(html):
    result = regx.search(__DATEFORMAT__, html, regx.IGNORECASE)
    if result is not None:
        return result[0][2]
    else:
        return '-'

def search(html):
    sID, sName = getIDandName(html, __SEARCHBUTTON__)
    print("sID:{0}  - sName: {1}".format(sID, sName))
    if sID is not "" or sName is not "":
        item = __BROWSER__.find_element_by_name(sName)
        item.click()
    else:
        __BROWSER__.switch_to.active_element.send_keys(Keys.ENTER)
    
def fillOutInformation(html, fn, ln, mn = "", dob = None):
    # NOTE: You have to fill them out in this order or sometimes
    # the open date selector messes with the submission
    fillOutDOB(html, dob)
    fillOutName(html, fn, ln, mn)

# Testing

In [161]:
__WEBSITEHTML__ = []

for site in __LEGALWEBSITES__:
    __WEBSITEHTML__.append(getSource(site))


Scraping website source code...
Scraping website source code...
Scraping website source code...
Scraping website source code...


In [279]:
cs = 0

In [290]:
html = __WEBSITEHTML__[cs]
current = __LEGALWEBSITES__[cs]
__BROWSER__.get(current)

fillOutInformation(html, __TESTING_FNAME__, __TESTING_LNAME__, __TESTING_MNAME__, __TESTING_DOB__)
search(html)

sID:  - sName: search


In [276]:
print(__WEBSITEHTML__[cs])


# Finding where to enter date of birth
# name=\"[^\"]*[Dd]o[Bb](irth)?[^\"]*

# Finding format for Date of Birth
# [Mm]+.[Dd]+.[Yy]+

<html lang="en-US" class=" js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths" style="height: 100%;"><!--<![endif]--><head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <title>
    OSCN Docket Search
  </title>
        <meta name="description" content="">
        <meta name="viewport" content="width=device-width, initial-scale=1">


        <link rel="stylesheet" href="/assets/css/normalize.min.css">
        <link rel="stylesheet" href="/assets/css/main.css">
        <link rel="stylesheet" href="/assets/css/bootstrap-2.3.2.min.css">
      

# ToDo
- Finish setting up date input

# Completed
- Input for f/m/l name